In [2]:
import mxnet as mx
from mxnet import nd

In [3]:
#set random seed
mx.random.seed(123)

In [7]:
#Create a 2D NDArray without any values initialized
# - Array has floats with arbitrary magnitude and sign
# - The array entries are float32 by default
x = nd.empty((3,4))

# Notice that the print statement states that a CPU is being used
# instead of GPU
print(x)


[[  0.00000000e+00   3.68934881e+19   0.00000000e+00   3.68934881e+19]
 [  1.55018351e-37   2.19544233e-40  -3.09922530e+26   2.29560714e-40]
 [  0.00000000e+00   3.68934881e+19  -1.98820301e+09   2.78373545e-40]]
<NDArray 3x4 @cpu(0)>


In [10]:
x = nd.zeros((3,5))
print(x)

x = nd.ones((3,4))
print(x)


[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
<NDArray 3x5 @cpu(0)>

[[ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]]
<NDArray 3x4 @cpu(0)>


In [12]:
#loc is the mean
#scale is standard deviation
y = nd.random_normal(loc = 0, scale = 1,shape = (3,4))
print(y)


[[-0.17074773  1.01052964  1.14151955  0.14561865]
 [ 0.4252629  -0.04724792 -0.26625201 -0.03935841]
 [ 1.75082421  0.82531595 -0.73186058  0.85916942]]
<NDArray 3x4 @cpu(0)>


In [15]:
print(y.shape)
print(y.size)

(3L, 4L)
12


In [21]:
#Standard Mathematical Operations

##Element-wise Addition
print(x+y)

##Element-wise Multiplication
x*y

##Exponentiation
print(nd.exp(y))

##Matrix Dot Product & Array Transpose
nd.dot(x,x.T)


[[ 0.82925224  2.01052952  2.14151955  1.14561868]
 [ 1.42526293  0.95275205  0.73374796  0.96064162]
 [ 2.75082421  1.82531595  0.26813942  1.85916948]]
<NDArray 3x4 @cpu(0)>

[[ 0.84303421  2.74705553  3.13152313  1.15675497]
 [ 1.52999258  0.95385087  0.76624602  0.96140605]
 [ 5.75934744  2.28260183  0.48101321  2.36119866]]
<NDArray 3x4 @cpu(0)>



[[ 4.  4.  4.]
 [ 4.  4.  4.]
 [ 4.  4.  4.]]
<NDArray 3x3 @cpu(0)>

In [30]:
#In-Place Operations with MXNET

##Not in-place operations
print('New Memory Allocated')
print('id(y):',id(y))
y = y+x
print('id(y):',id(y))

##In-place operation <but still makes use of a temporary buffer>
print('\nReuse Memory')
print('id(y): %s' %(id(y)))
y[:] = y+x
print('id(y): %s' %(id(y)))

##Use mdnet ndarray elemwise_add operation for in-place 
##operation without a buffer using the `out` keyword arg
print('\nid(y): %s' %(id(y)))
nd.elemwise_add(x,y,out = y)
print('id(y): %s' %(id(y)))


New Memory Allocated
('id(y):', 4532755280)
('id(y):', 4441626000)

Reuse Memory
id(y): 4441626000
id(y): 4441626000

id(y): 4441626000
id(y): 4441626000


In [34]:
#Converting mxnet ndarray to numpy
print(type(y))
print(type(y.asnumpy()))

<class 'mxnet.ndarray.ndarray.NDArray'>
<type 'numpy.ndarray'>


## Context Management

One of the key features that differentiates MXNet from NumPy 
is its support for diverse hardware devices.

In MXNet, every array has a context. One context could be the CPU. Other contexts might be various GPUs. Things can get even hairier when we deploy jobs across multiple servers. By assigning arrays to contexts intelligently, we can minimize the time spent transferring data between devices.

For example, when training neural networks on a server with a GPU, we typically prefer for the model’s parameters to live on the GPU. 

In [41]:
#The `ctx` parameter specifies the context
z = nd.ones(shape=(3,3), ctx=mx.cpu(0))
print(z)
print('\ncontext of z: %s' % (z.context))

##NOTE: Operations on multiple ndarrays requires them all
# to be on the same context. Transferring variables between 
# contexts is a slow process.


[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
<NDArray 3x3 @cpu(0)>

context of z: cpu(0)


In [46]:
print('id(z):', id(z))
z = z.copyto(mx.cpu(0))
print('id(z):', id(z))

#as_in_context(...) allocates new memory if the variable is
#not already present on the specified context
m = z.as_in_context(mx.cpu(0))
print('id(m):', id(m))
m = x.as_in_context(mx.cpu(0))
print('id(m):', id(m))
print(m)

('id(z):', 4540224720)
('id(z):', 4533493136)
('id(m):', 4533493136)
('id(m):', 4532754576)

[[ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]]
<NDArray 3x4 @cpu(0)>
